In [1]:
import pandas as pd
import random
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import files

uploaded = files.upload()

Saving food_data (2).csv to food_data (2).csv


In [3]:
dataset = pd.read_csv('food_data (2).csv', encoding='latin-1')
dataset

,food_name,food_weight,calories,unit,breakfast,lunch,dinner
0,Jagung Rebus,250,90.2,1.00,1,1,1
1,Kentang Rebus,200,166.0,2.00,1,1,1
2,Ketan Putih,120,217.0,2.75,1,1,0
3,Ketupat,160,32.0,0.50,1,1,0
4,Lontong,200,38.0,0.50,1,1,1
...,...,...,...,...,...,...,...
216,Risol,100,247.0,3.00,1,1,1
217,Roti Coklat,100,240.0,3.00,1,1,1
218,Sosis Solo,50,191.0,2.25,1,1,1
219,Sus Vla,85,129.0,1.50,1,1,1


In [4]:
dataset.shape

(221, 7)

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   food_name    221 non-null    object 
 1   food_weight  221 non-null    int64  
 2   calories     221 non-null    float64
 3   unit         221 non-null    float64
 4   breakfast    221 non-null    int64  
 5   lunch        221 non-null    int64  
 6   dinner       221 non-null    int64  
dtypes: float64(2), int64(4), object(1)
memory usage: 12.2+ KB


In [6]:
dataset.isnull().sum()

food_name      0
food_weight    0
calories       0
unit           0
breakfast      0
lunch          0
dinner         0
dtype: int64

In [7]:
dataset.duplicated().sum()

0

In [8]:
dataset.describe()

,food_weight,calories,unit,breakfast,lunch,dinner
count,221.000000,221.000000,221.000000,221.000000,221.000000,221.000000
mean,111.696833,183.222624,2.294118,0.556561,0.968326,0.769231
std,61.418412,127.580698,1.584201,0.497918,0.175529,0.422282
min,10.000000,18.000000,0.250000,0.000000,0.000000,0.000000
25%,70.000000,94.400000,1.250000,0.000000,1.000000,1.000000
50%,100.000000,147.000000,1.750000,1.000000,1.000000,1.000000
75%,150.000000,247.000000,3.000000,1.000000,1.000000,1.000000
max,420.000000,729.000000,9.000000,1.000000,1.000000,1.000000


In [9]:
corr = dataset.corr()
corr

<ipython-input-9-721137479888>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = dataset.corr()


,food_weight,calories,unit,breakfast,lunch,dinner
food_weight,1.000000,0.279401,0.260079,-0.132093,-0.020290,-0.137308
calories,0.279401,1.000000,0.980937,0.123747,0.061006,0.179819
unit,0.260079,0.980937,1.000000,0.132961,0.066347,0.193647
breakfast,-0.132093,0.123747,0.132961,1.000000,-0.161437,-0.099776
lunch,-0.020290,0.061006,0.066347,-0.161437,1.000000,0.207556
dinner,-0.137308,0.179819,0.193647,-0.099776,0.207556,1.000000


In [10]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
dataset['food_name'] = label_encoder.fit_transform(dataset['food_name'])

In [11]:
dataset['food_name'] = dataset['food_name'].astype(float)


In [12]:
X = dataset[['food_name', 'food_weight', 'calories', 'unit']]
y_breakfast = dataset['breakfast']
y_lunch = dataset['lunch']
y_dinner = dataset['dinner']
X_train, X_test, y_train_breakfast, y_test_breakfast, y_train_lunch, y_test_lunch, y_train_dinner, y_test_dinner = train_test_split(X, y_breakfast, y_lunch, y_dinner, test_size=0.25)


In [13]:
breakfast_classifier = DecisionTreeClassifier(criterion='entropy')
lunch_classifier = DecisionTreeClassifier(criterion='entropy')
dinner_classifier = DecisionTreeClassifier(criterion='entropy')


In [14]:
breakfast_classifier.fit(X_train, y_train_breakfast)
lunch_classifier.fit(X_train, y_train_lunch)
dinner_classifier.fit(X_train, y_train_dinner)

DecisionTreeClassifier(criterion='entropy')

In [15]:
def generate_diet_recommendation(X_test, total_calories):
    breakfast_calories = total_calories * 0.25
    lunch_calories = total_calories * 0.40
    dinner_calories = total_calories * 0.35

    breakfast_candidates = X_test[(X_test['calories'] <= breakfast_calories)]
    breakfast_predictions = breakfast_classifier.predict(breakfast_candidates)
    breakfast_recommendation = random.sample(list(breakfast_candidates.index[breakfast_predictions == 1]), min(5, len(breakfast_candidates)))

    lunch_candidates = X_test[(X_test['calories'] <= lunch_calories)]
    lunch_predictions = lunch_classifier.predict(lunch_candidates)
    lunch_recommendation = random.sample(list(lunch_candidates.index[lunch_predictions == 1]), min(5, len(lunch_candidates)))

    dinner_candidates = X_test[(X_test['calories'] <= dinner_calories)]
    dinner_predictions = dinner_classifier.predict(dinner_candidates)
    dinner_recommendation = random.sample(list(dinner_candidates.index[dinner_predictions == 1]), min(5, len(dinner_candidates)))

    return breakfast_recommendation, lunch_recommendation, dinner_recommendation

In [16]:
total_calories = 1500  # Example total calories
breakfast_recommendation, lunch_recommendation, dinner_recommendation = generate_diet_recommendation(X_test, total_calories)


In [17]:
# Evaluate the models
breakfast_accuracy = accuracy_score(y_test_breakfast, breakfast_classifier.predict(X_test))
breakfast_precision = precision_score(y_test_breakfast, breakfast_classifier.predict(X_test))
breakfast_recall = recall_score(y_test_breakfast, breakfast_classifier.predict(X_test))
breakfast_confusion_mat = confusion_matrix(y_test_breakfast, breakfast_classifier.predict(X_test))

lunch_accuracy = accuracy_score(y_test_lunch, lunch_classifier.predict(X_test))
lunch_precision = precision_score(y_test_lunch, lunch_classifier.predict(X_test))
lunch_recall = recall_score(y_test_lunch, lunch_classifier.predict(X_test))
lunch_confusion_mat = confusion_matrix(y_test_lunch, lunch_classifier.predict(X_test))

dinner_accuracy = accuracy_score(y_test_dinner, dinner_classifier.predict(X_test))
dinner_precision = precision_score(y_test_dinner, dinner_classifier.predict(X_test))
dinner_recall = recall_score(y_test_dinner, dinner_classifier.predict(X_test))
dinner_confusion_mat = confusion_matrix(y_test_dinner, dinner_classifier.predict(X_test))

In [18]:
print("Breakfast Classifier:")
print("Accuracy:", breakfast_accuracy)
print("Precision:", breakfast_precision)
print("Recall:", breakfast_recall)
print("Confusion Matrix:")
print(breakfast_confusion_mat)

print("\nLunch Classifier:")
print("Accuracy:", lunch_accuracy)
print("Precision:", lunch_precision)
print("Recall:", lunch_recall)
print("Confusion Matrix:")
print(lunch_confusion_mat)

print("\nDinner Classifier:")
print("Accuracy:", dinner_accuracy)
print("Precision:", dinner_precision)
print("Recall:", dinner_recall)
print("Confusion Matrix:")
print(dinner_confusion_mat)

Breakfast Classifier:
Accuracy: 0.7321428571428571
Precision: 0.71875
Recall: 0.7931034482758621
Confusion Matrix:
[[18  9]
 [ 6 23]]

Lunch Classifier:
Accuracy: 0.9464285714285714
Precision: 0.9629629629629629
Recall: 0.9811320754716981
Confusion Matrix:
[[ 1  2]
 [ 1 52]]

Dinner Classifier:
Accuracy: 0.8392857142857143
Precision: 0.8913043478260869
Recall: 0.9111111111111111
Confusion Matrix:
[[ 6  5]
 [ 4 41]]


In [19]:
print("\nBreakfast Recommendation:")
for item in breakfast_recommendation:
    print(item)

print("\nLunch Recommendation:")
for item in lunch_recommendation:
    print(item)

print("\nDinner Recommendation:")
for item in dinner_recommendation:
    print(item)


Breakfast Recommendation:
130
78
159
149
17

Lunch Recommendation:
49
28
220
46
216

Dinner Recommendation:
11
151
59
213
173
